<h1>Video Game Data EDA</h1>

In [253]:
import pandas as pd
import numpy as np

In [254]:
raw_games_data_df = pd.read_csv('../data/vgchartz_games_webscrape.csv', dtype=str)
expanded_games_data_df = pd.read_csv('../data/games_data_expanded.csv')

<h3>Prepping the data for merging</h3>

We are making necessary modifications in order to perform a merger between the two datasets.

In [255]:
# Reformat the release date column to be YYYY-MM-DD
raw_games_data_df['release_date'] = pd.to_datetime(raw_games_data_df['release_date'], errors='coerce').dt.strftime('%Y-%m-%d')
raw_games_data_df['last_update_date'] = pd.to_datetime(raw_games_data_df['last_update_date'], errors='coerce').dt.strftime('%Y-%m-%d')

# Rename the columns in the expanded dataset to match the raw dataset
expanded_games_data_df = expanded_games_data_df.rename(columns={
    'Name': 'game',
})

# Strip out any whitespace from the 'game' column
raw_games_data_df['game'] = raw_games_data_df['game'].str.strip()
expanded_games_data_df['game'] = expanded_games_data_df['game'].str.strip()

# Strip out any whitespace from the 'vgchartz_score' column
raw_games_data_df['vgchartz_score'] = raw_games_data_df['vgchartz_score'].str.strip()

<h3>Merging the 2 datasets</h3>

We are merging the valuable datapoints from the expanded dataset (sourced from Kaggle) into the original dataset (webscraped from VGChartz).

In [256]:
raw_games_data_df['metacritic_count'] = raw_games_data_df.merge(expanded_games_data_df, on='game', how='left')['Critic_Count']
raw_games_data_df['metacritic_score'] = raw_games_data_df.merge(expanded_games_data_df, on='game', how='left')['Critic_Score']
raw_games_data_df['metacritic_user_count'] = raw_games_data_df.merge(expanded_games_data_df, on='game', how='left')['User_Count']
raw_games_data_df['metacritic_user_score'] = raw_games_data_df.merge(expanded_games_data_df, on='game', how='left')['User_Score']
raw_games_data_df['esrb_rating'] = raw_games_data_df.merge(expanded_games_data_df, on='game', how='left')['Rating']
raw_games_data_df['genre'] = raw_games_data_df.merge(expanded_games_data_df, on='game', how='left')['Genre']

<h3>Exploratory data analysis findings</h3>

&#x2022; `critic_score` and `user_score` are wildly inconsistent in their appearances. Thus, we will ignore and drop these columns.

&#x2022; `total_shipped` represents volume of sales, not revenue or profit.

&#x2022; Unfortunately, the individual breakdown of the sales by geographical region is not available. Perhaps I can supplement this data with a different dataset as a stretch goal.

&#x2022; `last_update_date` is essentially useless for this project. We will drop this column.

&#x2022; With the remaining columns, I will be able to glean enough information to answer the initial question posed - at least on a surface level. More data will be needed to answer the question in more depth.

&#x2022; If the `total_shipped` column is empty, but the `total_sales` column is not, we will update the `total_shipped` column with the value in the `total_sales` column.

<h3>Data cleaning and manipulation</h3>

In [257]:
# Replace the null values in total_shipped with the values in total_sales
raw_games_data_df['total_shipped'] = np.where(raw_games_data_df['total_shipped'].isnull() & raw_games_data_df['total_sales'].notnull(), raw_games_data_df['total_sales'], raw_games_data_df['total_shipped'])
gta_game_df = raw_games_data_df[raw_games_data_df['game'] == 'Grand Theft Auto V']
n_games_without_sales_data = raw_games_data_df['total_shipped'].isnull().sum()

print(f'There are {n_games_without_sales_data} games without sales data. We will drop these records now.')

# Drop the rows where total_shipped is null
raw_games_data_df = raw_games_data_df.dropna(subset=['total_shipped'])

print(f'The total number of games in the dataset is now {len(raw_games_data_df)}.')

# Drop unnecessary columns
try:
    raw_games_data_df = raw_games_data_df.drop(
        columns=[
            'last_update_date',
            'critic_score',
            'user_score'
        ]
    )
except KeyError:
    print('Unnecessary columns have already been dropped.')

# Strip out the 'm' from the end of the total_shipped column
try:
    raw_games_data_df['total_shipped'] = raw_games_data_df['total_shipped'].str.replace('m', '')
except AttributeError:
    print('The total_shipped column has already been converted to a float.')

# Turn the total_shipped column into a float
raw_games_data_df['total_shipped'] = raw_games_data_df['total_shipped'].astype(float)

# Remove all rows where the total_shipped value is 0 (less than 100,000 is the true value)
raw_games_data_df = raw_games_data_df[raw_games_data_df['total_shipped'] > 0]

print(f'We are dropping the games that sold less than 100,000. The total number of games in the dataset is now {len(raw_games_data_df)}.')

# Remove 'tbd' values from the metacritic_user_score column
try:
    raw_games_data_df = raw_games_data_df[raw_games_data_df['metacritic_user_score'] != 'tbd']
except TypeError:
    print('The tbd values have already been removed from the metacritic_user_score column.')

# Multiply the metacritic_user_score column by 10 to get the correct value (1-100)
raw_games_data_df['metacritic_user_score'] = raw_games_data_df['metacritic_user_score'].astype(float) * 10

There are 39613 games without sales data. We will drop these records now.
The total number of games in the dataset is now 23247.
We are dropping the games that sold less than 100,000. The total number of games in the dataset is now 21881.


<h3>Write scrubbed data to csv</h3>

In [258]:
# Write the cleaned dataset to a csv file
raw_games_data_df.to_csv('../data/normalized_games_data.csv', index=False)